In [1]:
from collections import OrderedDict
from typing import List, Tuple, Dict, Optional

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

import flwr as fl
from flwr.common import Metrics
import flwr.common


DEVICE = torch.device("cuda")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

NUM_CLIENTS = 10

BATCH_SIZE = 32

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)
trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
testset = CIFAR10("./dataset", train=False, download=True, transform=transform)



lengths = [45000, 5000]
split_trainset, valset = random_split(trainset, lengths, torch.Generator().manual_seed(42)) 
    
# 5000
full_valloader = DataLoader(valset, batch_size=BATCH_SIZE, shuffle=True)

# 45000
full_split_trainloader = DataLoader(split_trainset, batch_size=BATCH_SIZE, shuffle=True)
# 10000
full_testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True)


def load_datasets():
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)



    lengths = [45000, 5000]
    split_trainset, valset = random_split(trainset, lengths, torch.Generator().manual_seed(42)) 
    
    # 5000
    full_valset = DataLoader(valset, batch_size=BATCH_SIZE, shuffle=True)

    # 45000
    full_split_trainloader = DataLoader(split_trainset, batch_size=BATCH_SIZE, shuffle=True)
    # 10000
    full_testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True)

    # Split training set into 10 partitions to simulate the individual dataset
    partition_size = len(trainset) // NUM_CLIENTS
    lengths = [partition_size] * NUM_CLIENTS
    datasets= random_split(trainset, lengths, torch.Generator().manual_seed(42))

    

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for idx, ds in enumerate(datasets):
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        # Always splits in the same way.

        # print(dict(Counter(ds_train.targets)))
        # custom_subset(ds_train)
        # arr = []
        # print("ds_train", idx)
        # for a in ds_train:
        #     arr.append(a[1])
        # for i in range(10):
        #     print(i, ":", arr.count(i))
        # Data is not perfectly distributed


        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    return trainloaders, valloaders, testloader


trainloaders, valloaders, testloader = load_datasets()


Training on cuda using PyTorch 2.0.0 and Flower 1.4.0
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [2]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
def train(net, trainloader, epochs: int, verbose=False):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        if verbose:
            print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy



client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}



In [8]:
EPOCH = 100

In [11]:
import os
# 여기서는 10개로 나눈 것 중, 하나만 train 함.
import pandas as pd
df_final = pd.DataFrame()
print("Experiment on centralized manner.")

net = Net().to(DEVICE)
trained_path = "./dataset/trained_centralized.pkl"


if not (os.path.isfile(trained_path)):
    for epoch in range(EPOCH+1):
        train(net, full_split_trainloader, 1)
        loss, accuracy = test(net, full_valloader)
        print(f"Epoch {epoch+1}: validation loss {loss}, accuracy {accuracy}")
        df_result = pd.DataFrame()
        df_result['round'] = epoch+1,
        df_result['strategy'] = 'Central',
        df_result['c_loss'] = loss,
        df_result['c_accuracy'] = accuracy,
        df_result['d_accuracy'] = 0.0

        df_final = pd.concat([df_final, df_result], axis=0)

    torch.save(net.state_dict(), trained_path)
else :
    net.load_state_dict(torch.load(trained_path))
loss, accuracy = test(net, full_testloader)
print(f"Final test set performance:\n\tloss {loss}\n\taccuracy {accuracy}")

df_final

Experiment on centralized manner.
Epoch 1: validation loss 0.04441600443124771, accuracy 0.4778
Epoch 2: validation loss 0.03931218582391739, accuracy 0.5474
Epoch 3: validation loss 0.037541953146457675, accuracy 0.5656
Final test set performance:
	loss 0.037185421848297116
	accuracy 0.5761


,round,strategy,c_loss,c_accuracy,d_accuracy
0,0,Central,0.044416,0.4778,0.0
0,1,Central,0.039312,0.5474,0.0
0,2,Central,0.037542,0.5656,0.0


In [10]:
df_final

""


In [5]:
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

class FlowerClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, valloader):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        return get_parameters(self.net)

    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        print(f"Accuracy {accuracy}")
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}
    
def client_fn(cid: str) -> FlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    net = Net().to(DEVICE)

    # Load data (CIFAR-10)
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    # Create a  single Flower client representing a single organization
    return FlowerClient(net, trainloader, valloader)
    
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

# The `evaluate` function will be by Flower called after every round
def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = Net().to(DEVICE)
    valloader = valloaders[0]
    set_parameters(net, parameters)  # Update model with the latest parameters
    loss, accuracy = test(net, valloader)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}


In [6]:
fedavg = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    fraction_evaluate=0.5,
    min_fit_clients=10,
    min_evaluate_clients=5,
    min_available_clients=10,
    evaluate_metrics_aggregation_fn=weighted_average,   # aggregate evaluation of local model
    evaluate_fn=evaluate,   # evaluate global model
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
)

fedavgM = fl.server.strategy.FedAvgM(
    fraction_fit=1.0,
    fraction_evaluate=0.5,
    min_fit_clients=10,
    min_evaluate_clients=5,
    min_available_clients=10,
    evaluate_metrics_aggregation_fn=weighted_average,   # aggregate evaluation of local model
    evaluate_fn=evaluate,   # evaluate global model
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
)

# qfedavg = fl.server.strategy.QFedAvg(
#     fraction_fit=1.0,
#     fraction_evaluate=0.5,
#     min_fit_clients=10,
#     min_evaluate_clients=5,
#     min_available_clients=10,
#     evaluate_metrics_aggregation_fn=weighted_average,   # aggregate evaluation of local model
#     evaluate_fn=evaluate,   # evaluate global model
#     initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
# )

# ftfedavg = fl.server.strategy.FaultTolerantFedAvg(
#     fraction_fit=1.0,
#     fraction_evaluate=0.5,
#     min_fit_clients=10,
#     min_evaluate_clients=5,
#     min_available_clients=10,
#     evaluate_metrics_aggregation_fn=weighted_average,   # aggregate evaluation of local model
#     evaluate_fn=evaluate,   # evaluate global model
#     initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
# )

# fedopt = fl.server.strategy.FedOpt(
#     fraction_fit=1.0,
#     fraction_evaluate=0.5,
#     min_fit_clients=10,
#     min_evaluate_clients=5,
#     min_available_clients=10,
#     evaluate_metrics_aggregation_fn=weighted_average,   # aggregate evaluation of local model
#     evaluate_fn=evaluate,   # evaluate global model
#     initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
# )

# fedprox = fl.server.strategy.FedProx(
#     fraction_fit=1.0,
#     fraction_evaluate=0.5,
#     min_fit_clients=10,
#     min_evaluate_clients=5,
#     min_available_clients=10,
#     evaluate_metrics_aggregation_fn=weighted_average,   # aggregate evaluation of local model
#     evaluate_fn=evaluate,   # evaluate global model
#     initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
#     proximal_mu=0.1,
# )

# fedadagrad = fl.server.strategy.FedAdagrad(
#     fraction_fit=1.0,
#     fraction_evaluate=0.5,
#     min_fit_clients=10,
#     min_evaluate_clients=5,
#     min_available_clients=10,
#     evaluate_metrics_aggregation_fn=weighted_average,   # aggregate evaluation of local model
#     evaluate_fn=evaluate,   # evaluate global model
#     initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
# )

# fedadam = fl.server.strategy.FedAdam(
#     fraction_fit=1.0,
#     fraction_evaluate=0.5,
#     min_fit_clients=10,
#     min_evaluate_clients=5,
#     min_available_clients=10,
#     evaluate_metrics_aggregation_fn=weighted_average,   # aggregate evaluation of local model
#     evaluate_fn=evaluate,   # evaluate global model
#     initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
# )

# fedyogi = fl.server.strategy.FedYogi(
#     fraction_fit=1.0,
#     fraction_evaluate=0.5,
#     min_fit_clients=10,
#     min_evaluate_clients=5,
#     min_available_clients=10,
#     evaluate_metrics_aggregation_fn=weighted_average,   # aggregate evaluation of local model
#     evaluate_fn=evaluate,   # evaluate global model
#     initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
# )

In [7]:

strategies = {
    'FedAvg': fedavg,
    'FedAvgM': fedavgM,
    # 'QFedAvg': qfedavg,
    # 'FaultTolerantFedAvg': ftfedavg,
    # 'FedOpt': fedopt,
    # 'FedProx': fedprox,
    # 'FedAdagrad': fedadagrad,
    # 'FedAdam': fedadam,
    # 'FedYogi': fedyogi,
}

print("Experiment on federated manner.")
for sname, strategy in strategies.items():
    print(f"{sname} simulation")

    hist = fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=NUM_CLIENTS,
        config=fl.server.ServerConfig(num_rounds=EPOCH),
        strategy=strategy,
        client_resources=client_resources,
    )

    df_result = pd.DataFrame()
    df_result['round'] = [i for i in range(1, EPOCH + 1)]
    df_result['strategy'] = sname

    # centralized metrics
    metrics_cen = list(hist.metrics_centralized.keys())
    metrics_dis = list(hist.metrics_distributed.keys())

    for metric in metrics_cen:
        df_result[f"c_{metric}"] = [h[1] for h in hist.metrics_centralized[metric][1:]]
    for metric in metrics_dis:
        df_result[f"d_{metric}"] = [h[1] for h in hist.metrics_distributed[metric]]

    df_final = pd.concat([df_final, df_result], axis=0)

df_final.to_csv('./result/result.csv', index=False)

INFO flwr 2023-05-02 10:28:45,703 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=2, round_timeout=None)


Experiment on federated manner.
FedAvg simulation


2023-05-02 10:28:47,448	INFO worker.py:1553 -- Started a local Ray instance.
INFO flwr 2023-05-02 10:28:48,043 | app.py:180 | Flower VCE: Ray initialized with resources: {'object_store_memory': 23141836800.0, 'GPU': 1.0, 'memory': 46283673600.0, 'CPU': 16.0, 'node:172.17.0.2': 1.0, 'accelerator_type:RTX': 1.0}
INFO flwr 2023-05-02 10:28:48,044 | server.py:86 | Initializing global parameters
INFO flwr 2023-05-02 10:28:48,044 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2023-05-02 10:28:48,045 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-05-02 10:28:48,119 | server.py:91 | initial parameters (loss, other metrics): 0.07364347982406616, {'accuracy': 0.102}
INFO flwr 2023-05-02 10:28:48,119 | server.py:101 | FL starting
DEBUG flwr 2023-05-02 10:28:48,119 | server.py:218 | fit_round 1: strategy sampled 10 clients (out of 10)


Server-side evaluation loss 0.07364347982406616 / accuracy 0.102


DEBUG flwr 2023-05-02 10:29:14,447 | server.py:232 | fit_round 1 received 10 results and 0 failures
WARNING flwr 2023-05-02 10:29:14,459 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
INFO flwr 2023-05-02 10:29:14,534 | server.py:119 | fit progress: (1, 0.0641543595790863, {'accuracy': 0.304}, 26.41476331499871)
DEBUG flwr 2023-05-02 10:29:14,535 | server.py:168 | evaluate_round 1: strategy sampled 5 clients (out of 10)


...

(launch_and_evaluate pid=1724429) Accuracy 0.316


DEBUG flwr 2023-05-02 10:31:08,057 | server.py:232 | fit_round 2 received 10 results and 0 failures
INFO flwr 2023-05-02 10:31:08,142 | server.py:119 | fit progress: (2, 0.05581475305557251, {'accuracy': 0.352}, 62.83309047098737)
DEBUG flwr 2023-05-02 10:31:08,142 | server.py:168 | evaluate_round 2: strategy sampled 5 clients (out of 10)


Server-side evaluation loss 0.05581475305557251 / accuracy 0.352
(launch_and_evaluate pid=1725693) Accuracy 0.362
(launch_and_evaluate pid=1725808) Accuracy 0.352
(launch_and_evaluate pid=1725922) Accuracy 0.356
(launch_and_evaluate pid=1726037) Accuracy 0.352


DEBUG flwr 2023-05-02 10:31:18,089 | server.py:182 | evaluate_round 2 received 5 results and 0 failures
INFO flwr 2023-05-02 10:31:18,089 | server.py:147 | FL finished in 72.78040773805697
INFO flwr 2023-05-02 10:31:18,090 | app.py:218 | app_fit: losses_distributed [(1, 0.06277724905014037), (2, 0.05550191216468812)]
INFO flwr 2023-05-02 10:31:18,090 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-05-02 10:31:18,090 | app.py:220 | app_fit: metrics_distributed {'accuracy': [(1, 0.312), (2, 0.35439999999999994)]}
INFO flwr 2023-05-02 10:31:18,090 | app.py:221 | app_fit: losses_centralized [(0, 0.07371985340118409), (1, 0.06301632738113404), (2, 0.05581475305557251)]
INFO flwr 2023-05-02 10:31:18,091 | app.py:222 | app_fit: metrics_centralized {'accuracy': [(0, 0.096), (1, 0.292), (2, 0.352)]}


(launch_and_evaluate pid=1726157) Accuracy 0.35
